In [1]:
import pickle
import pathlib
import itertools

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
import shapely
from tqdm.auto import tqdm


/Users/baart_f/src/fis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir_euris = pathlib.Path('~/data/euris').expanduser()
data_dir_eurostat = pathlib.Path('/Users/baart_f/data/eurostat').expanduser()

In [3]:
with (data_dir_euris / 'export-graph-v0.1.0.pickle').open('rb') as f:
    graph = pickle.load(f)


In [4]:
nodes_df = pd.DataFrame(graph.nodes.values(), index=graph.nodes.keys())
nodes_gdf = gpd.GeoDataFrame(nodes_df, crs='EPSG:4326')

nodes_gdf = nodes_gdf[nodes_gdf['subgraph'] == 2]
nodes_gdf.head()


,euris_nodes,objectcode_cb,hectom_cb,sectionref_cb,locode_cb,function,ww_name,ww_name_cb,rt_name,rt_name_cb,...,sectionref,hectom,vplnpoint,path,countrycode_locode,countrycode_path,countrycode,node_id,geometry,subgraph
SK_J0018,"[{'objectcode_cb': None, 'hectom_cb': '17840',...",None,17840,HU0000100020,HUXXX00001J003317840,junction,Dunaj,Duna,Dunaj,Duna,...,SK0000100016,17840,2.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0018,POINT (17.91978 47.74429),2
SK_J0019,"[{'objectcode_cb': 'J0032', 'hectom_cb': '1767...",J0032,17671,HU0000100018,HUXXX00001J003217671,junction,Dunaj,Duna,Dunaj,Duna,...,SK0000100013,17671,0.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0019,POINT (18.13045 47.75027),2
SK_J0006,"[{'objectcode_cb': 'J0031', 'hectom_cb': '1765...",J0031,17658,HU0000100017,HUXXX00001J003117658,junction,Dunaj,Duna,Dunaj,Duna,...,SK0000100014,17658,0.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0006,POINT (18.14739 47.74796),2
SK_J0007,"[{'objectcode_cb': None, 'hectom_cb': '17160',...",None,17160,HU0000100015,HUXXX00001J003017160,junction,Dunaj,Duna,Dunaj,Duna,...,SK0000100015,17160,2.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0007,POINT (18.74512 47.8144),2
SK_J0008,"[{'objectcode_cb': 'J0018', 'hectom_cb': '1708...",J0018,17083,HU0000100015,HUXXX00001J001817083,junction,Dunaj,Duna,Dunaj,Duna,...,SK0000100015,17082,1.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0008,POINT (18.84824 47.81609),2


In [5]:
nodes_gdf['node_geometry'] = nodes_gdf['geometry']

In [6]:
nuts_gdf = gpd.read_file(data_dir_eurostat / 'NUTS2_centroid.gpkg')


In [7]:
UTM_31N = 'EPSG:32631'
nuts_nodes_gdf = (
    nuts_gdf
        .to_crs(UTM_31N)
        .sjoin_nearest(
            nodes_gdf.to_crs(UTM_31N)
        )
).to_crs('EPSG:4326')

In [8]:
nuts  = 'DEA2'
n = 'DE_J3238'

In [9]:
nuts_nodes_gdf[nuts_nodes_gdf['NUTS_ID'] == nuts]['node_id'] == n

283    True
Name: node_id, dtype: bool

In [10]:
nuts_nodes_gdf['connection'] = nuts_nodes_gdf.apply(
    lambda row: shapely.LineString([row['geometry'], row['node_geometry']]),
    axis=1
)
(
    nuts_nodes_gdf
        .set_geometry('connection')
        .drop(columns=['geometry', 'node_geometry'])
        .set_crs('EPSG:4326')
        .to_file(data_dir_eurostat / 'connected-nuts-nodes.gpkg')
)

In [11]:
nuts_nodes_gdf

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry,index_right,...,hectom,vplnpoint,path,countrycode_locode,countrycode_path,countrycode,node_id,subgraph,node_geometry,connection
0,ITC4,2,IT,Lombardia,Lombardia,NaN,NaN,NaN,POINT (9.77652 45.62073),FR_J1692,...,1683,1.0,Node_FR_20241106.geojson,FR,FR,FR,FR_J1692,2,POINT (7.58484 47.57515),"LINESTRING (9.77652 45.62073, 7.58484 47.57515)"
1,ITF1,2,IT,Abruzzo,Abruzzo,NaN,NaN,NaN,POINT (13.85093 42.2278),AT_J0071,...,20940,0.0,Node_AT_20250512.geojson,AT,AT,AT,AT_J0071,2,POINT (14.70561 48.16507),"LINESTRING (13.85093 42.2278, 14.70561 48.16507)"
2,ITF2,2,IT,Molise,Molise,NaN,NaN,NaN,POINT (14.59291 41.68755),AT_J0071,...,20940,0.0,Node_AT_20250512.geojson,AT,AT,AT,AT_J0071,2,POINT (14.70561 48.16507),"LINESTRING (14.59291 41.68755, 14.70561 48.16507)"
3,ITF3,2,IT,Campania,Campania,NaN,NaN,NaN,POINT (14.83226 40.86386),SK_J0022,...,17938,0.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0022,2,POINT (17.79224 47.74065),"LINESTRING (14.83226 40.86386, 17.79224 47.74065)"
4,ITF4,2,IT,Puglia,Puglia,NaN,NaN,NaN,POINT (16.61421 40.98666),SK_J0022,...,17938,0.0,Node_HU_20241118.geojson,SK,HU,SK,SK_J0022,2,POINT (17.79224 47.74065),"LINESTRING (16.61421 40.98666, 17.79224 47.74065)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,DEE0,2,DE,Sachsen-Anhalt,Sachsen-Anhalt,NaN,NaN,NaN,POINT (11.70704 52.00571),DE_J3093,...,00200,NaN,Node_DE_20250521.geojson,DE,DE,DE,DE_J3093,2,POINT (11.78915 51.90331),"LINESTRING (11.70704 52.00571, 11.78915 51.90331)"
295,DEF0,2,DE,Schleswig-Holstein,Schleswig-Holstein,NaN,NaN,NaN,POINT (9.81601 54.17491),DE_J0410,...,00000,NaN,Node_DE_20250521.geojson,DE,DE,DE,DE_J0410,2,POINT (10.64624 53.84405),"LINESTRING (9.81601 54.17491, 10.64624 53.84405)"
296,DEG0,2,DE,Thüringen,Thüringen,NaN,NaN,NaN,POINT (11.0212 50.90456),DE_J3094,...,00880,NaN,Node_DE_20250521.geojson,DE,DE,DE,DE_J3094,2,POINT (11.94532 51.52158),"LINESTRING (11.0212 50.90456, 11.94532 51.52158)"
297,DK01,2,DK,Hovedstaden,Hovedstaden,NaN,NaN,NaN,POINT (12.87333 55.6936),DE_J3110,...,00056,NaN,Node_DE_20250521.geojson,DE,DE,DE,DE_J3110,2,POINT (10.69085 53.87525),"LINESTRING (12.87333 55.6936, 10.69085 53.87525)"


In [19]:
hb_matrices_df = pd.read_csv(data_dir_eurostat / 'hb_matrices_rijn.csv')

In [20]:

hb_matrices_nodes_df = (
    hb_matrices_df
        .merge(
            nuts_nodes_gdf.set_index('NUTS_ID')[['node_id', 'node_geometry']], 
            left_on='nuts2_laad', 
            right_index=True
        ).rename(
            columns={
                'node_id': 'node_id_laad', 
                'node_geometry': 'node_geometry_laad'
            }
        ).merge(
            nuts_nodes_gdf.set_index('NUTS_ID')[['node_id', 'node_geometry']],
            left_on='nuts2_los',
            right_index=True
        ).rename(
            columns={
                'node_id': 'node_id_los', 
                'node_geometry': 'node_geometry_los'
            }
        )
)
hb_matrices_nodes_df['geometry'] = hb_matrices_nodes_df.apply(
    lambda row: shapely.LineString([row['node_geometry_laad'], row['node_geometry_los']]),
    axis=1
)
hb_matrices_nodes_gdf = gpd.GeoDataFrame(hb_matrices_nodes_df, crs='EPSG:4326')
hb_matrices_nodes_gdf.drop(columns=['node_geometry_laad', 'node_geometry_los']).to_file(data_dir_eurostat / 'hb_matrices_nodes.gpkg')


In [21]:
import pyproj
geod = pyproj.Geod(ellps='WGS84')

for edge in graph.edges.values():
    length_m = geod.geometry_length(edge['geometry'])
    edge['length_m'] = length_m

In [22]:
a = 'AT_J0042'	
b = 'BE_F5894'
def weight(a_i, b_i, edge):
    weight = edge['length_m']
    cemt = edge.get('cemt')
    if cemt not in {'IV', 'VII', 'VIa', 'VIb', 'VIc', 'Va', 'Vb'}:
        weight *= 10
    return weight
nx.shortest_path(graph, a, b, weight=weight);

In [23]:
segments = []

for idx, row in tqdm(hb_matrices_nodes_gdf.iterrows()):
    a = row['node_id_laad']
    b = row['node_id_los'] 
    path = nx.shortest_path(graph, a, b, weight=weight)
    linestrings = []
    for a_i, b_i in itertools.pairwise(path):
        # combine a,b and b,a into a,b
        a_i, b_i = sorted([a_i, b_i])
        edge = graph.edges[(a_i, b_i)]
        geometry = edge['geometry']
        linestrings.append(geometry)
        segment = {
            "geometry": geometry, 
            "path": path, 
            'index': idx, 
            "a": a_i, 
            "b": b_i, 
            "nuts2_laad": row['nuts2_laad'], 
            "nuts2_los": row['nuts2_los'],
            "gewicht": row['gewicht'],
            "ccr2025": row['ccr2025'],
            "jaar": row['jaar']
        }
        segments.append(segment)
hb_routes_df = pd.DataFrame(segments)
hb_routes_gdf = gpd.GeoDataFrame(hb_routes_df, crs='EPSG:4326')

# hb_routes_df = hb_matrices_nodes_gdf.apply(path, axis=1)


262242it [06:23, 684.45it/s] 


In [24]:
hb_routes_gdf

,geometry,path,index,a,b,nuts2_laad,nuts2_los,gewicht,ccr2025,jaar
0,"LINESTRING (15.649 48.39793, 15.64932 48.39767...","[AT_J0042, AT_J0002, AT_J0074, AT_J0040, AT_J0...",0,AT_J0002,AT_J0042,AT12,DE23,1437.0,Agribulk,1993
1,"LINESTRING (15.33121 48.23629, 15.33344 48.237...","[AT_J0042, AT_J0002, AT_J0074, AT_J0040, AT_J0...",0,AT_J0002,AT_J0074,AT12,DE23,1437.0,Agribulk,1993
2,"LINESTRING (15.3016 48.22598, 15.30383 48.2262...","[AT_J0042, AT_J0002, AT_J0074, AT_J0040, AT_J0...",0,AT_J0040,AT_J0074,AT12,DE23,1437.0,Agribulk,1993
3,"LINESTRING (15.09759 48.1721, 15.09905 48.1717...","[AT_J0042, AT_J0002, AT_J0074, AT_J0040, AT_J0...",0,AT_J0038,AT_J0040,AT12,DE23,1437.0,Agribulk,1993
4,"LINESTRING (15.07383 48.19007, 15.07449 48.189...","[AT_J0042, AT_J0002, AT_J0074, AT_J0040, AT_J0...",0,AT_J0037,AT_J0038,AT12,DE23,1437.0,Agribulk,1993
...,...,...,...,...,...,...,...,...,...,...
25310976,"LINESTRING (4.45089 51.90874, 4.45584 51.90809...","[SK_J0020, SK_J0002, AT_J0011, AT_J0010, AT_J0...",287388,NL_J1677,NL_J2083,SK01,NL36,290.0,Other Dry Bulk,2024
25310977,"LINESTRING (4.44996 51.90944, 4.45089 51.90874)","[SK_J0020, SK_J0002, AT_J0011, AT_J0010, AT_J0...",287388,NL_J1677,NL_J2082,SK01,NL36,290.0,Other Dry Bulk,2024
25310978,"LINESTRING (4.43234 51.92607, 4.43337 51.92527...","[SK_J0020, SK_J0002, AT_J0011, AT_J0010, AT_J0...",287388,NL_J2081,NL_J2082,SK01,NL36,290.0,Other Dry Bulk,2024
25310979,"LINESTRING (4.43234 51.92607, 4.44547 51.93345...","[SK_J0020, SK_J0002, AT_J0011, AT_J0010, AT_J0...",287388,NL_J2081,NL_J2148,SK01,NL36,290.0,Other Dry Bulk,2024


In [25]:
hb_routes_total_gdf = (
    hb_routes_gdf
        .groupby(['a', 'b', 'jaar'])
        .agg({'geometry': 'first', 'gewicht': 'sum'})
        .reset_index()
        .set_geometry('geometry')
        .set_crs('EPSG:4326')
)

hb_routes_total_gdf.to_file(data_dir_eurostat / 'results' / 'hb_routes_total-per-year.gpkg')

In [26]:
hb_routes_per_group_gdf = (
    hb_routes_gdf
        .groupby(['a', 'b', 'ccr2025', 'jaar'])
        .agg({'geometry': 'first', 'gewicht': 'sum'})
        .reset_index()
        .set_geometry('geometry')
        .set_crs('EPSG:4326')
)

hb_routes_per_group_gdf.to_file(data_dir_eurostat / 'results' / 'hb_routes_total-per-group.gpkg')